In [1]:
import pandas as pd
import numpy as np
import math
from scipy import signal

import tqdm
from tqdm.notebook import tqdm
import time

import pingouin as pg

import statistics

#現在の最大表示列数の出力
pd.get_option("display.max_columns")

#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 50)

pd.get_option("display.max_rows")
pd.set_option("display.max_rows",100)

# データの読み込み

In [3]:
# read expdata_doc
df_fm = pd.read_csv('../../dataset_create/expdata_doc.csv')
df_fm.head()

,Unnamed: 0,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,...,ete_larg_deviation_ave,20_ete_line_ave,50_ete_line_ave,80_ete_line_ave,ave_direction_ave,length_trajectory_ave,ratio_ete_ave,ave_v_ave,5points_m_acc_ave,m_stroke_press_ave,m_stroke_area_cover_ave,finger_orien_ave,cd_finger_orien_ave,phone_orien_ave,2stroke_a,2stroke_distance,2stroke_time,2stroke_v,a_stroke_inter,d_stroke_inter,outside_a,outside_d,outside_v,v_stroke_inter,multi_flag
0,0,27,6,2.118,1.398,6.5516,-36.286,7.1563,-15.825,20.4700,0.90037,3,1.5412,9.0436,14.3650,25.879,-252.21,-5.8162,233.88,7.3135,1.30430,0.205970,0.55933,9.644200e-01,1.5704,...,1.131850,0.258005,0.482420,0.840140,1.47840,17.56500,0.945310,15.3965,16.083,0.520,0.044444,0.0,0.0,1.0,6.690606,60.335949,3.003,20.091891,52.205705,27.063437,0.946163,8.532505,2.841327,37.588107,33
1,1,27,6,2.142,0.885,12.0950,-42.434,11.8940,-29.633,12.8020,0.80483,3,1.5865,6.5276,13.1470,25.325,-481.51,-56.4670,427.62,6.5735,0.95940,0.310040,0.40551,7.158600e-01,1.3864,...,0.627065,0.088795,0.259100,0.475475,1.34190,11.51275,0.937925,12.6055,134.970,0.550,0.044444,0.0,0.0,1.0,3.186922,30.626319,3.100,9.879458,5.717478,9.033896,1.324906,12.732347,4.107209,7.186870,33
2,2,27,6,2.943,0.958,13.6070,-38.503,13.0020,-29.734,8.7899,0.71704,3,1.6397,0.0000,8.3994,18.372,-405.22,0.0000,265.58,3.8767,0.29473,-0.132450,0.11269,2.350900e-01,1.2974,...,-0.444235,-0.611825,-0.257180,0.117545,1.38445,10.53775,0.934190,12.4525,461.530,0.530,0.044444,0.0,0.0,1.0,2.073913,28.653367,3.717,7.708735,2.277958,8.975666,0.792000,10.942343,2.943864,4.521746,33
3,3,27,6,1.537,0.774,14.9170,-38.503,14.7160,-27.617,10.8880,0.86631,3,1.5893,7.4564,11.9870,20.518,-491.98,-48.9550,624.62,9.3508,-1.18320,-1.091200,-0.62705,2.498000e-17,1.4715,...,-0.452005,-0.562763,-0.254025,0.112130,1.48550,10.88700,0.906030,14.1505,305.025,0.495,0.044444,0.0,0.0,1.0,5.925048,31.370763,2.301,13.633535,19.931940,11.603758,1.609201,8.520078,3.702772,15.208070,33
4,4,27,6,1.510,0.764,16.9330,-39.007,16.5300,-30.137,8.8790,0.69747,3,1.6162,5.1193,10.3960,31.476,-726.69,-57.9480,1072.40,6.2161,0.27919,-0.034326,0.11900,2.242600e-01,1.4995,...,0.295425,-0.093408,0.065065,0.249595,1.46490,8.41280,0.913520,11.0941,-317.765,0.510,0.044444,0.0,0.0,1.0,4.571208,23.347901,2.260,10.330930,14.518423,8.079735,1.436157,7.335317,3.245716,10.830743,33


In [4]:
df_ffc = pd.read_csv('expdata_fu_doc.csv')
df_ffc.head()

,Unnamed: 0,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,...,ete_larg_deviation_ave,20_ete_line_ave,50_ete_line_ave,80_ete_line_ave,ave_direction_ave,length_trajectory_ave,ratio_ete_ave,ave_v_ave,5points_m_acc_ave,m_stroke_press_ave,m_stroke_area_cover_ave,finger_orien_ave,cd_finger_orien_ave,phone_orien_ave,2stroke_a,2stroke_distance,2stroke_time,2stroke_v,a_stroke_inter,d_stroke_inter,outside_a,outside_d,outside_v,v_stroke_inter,multi_flag
0,0,1.0,3.0,3.371,1.620,25.072961,-37.391416,26.926180,-28.997425,8.596134,0.784513,3.0,1.353503,0.000000,6.412537,22.981771,-89.228250,5.329094e-12,377.208959,3.406645,6.649785,-5.777682,-1.253648,-0.381545,1.044725,...,8.612017,-0.708584,3.352146,4.660300,1.270401,11.299389,0.952447,7.663386,32.986789,0.640,0.188889,0.0,0.0,1.0,1.309492,29.694878,4.762,6.235800,2.661197,8.159233,1.270477,28.810146,6.050010,4.659756,33
1,1,1.0,3.0,2.062,1.391,30.087554,-21.475536,27.907296,-8.721030,12.939511,0.741063,3.0,1.740100,3.806625,7.267474,18.168866,-528.988537,1.048354e+01,421.465440,1.927095,10.574249,4.360515,7.957940,9.702146,1.496077,...,9.484120,2.943348,6.404506,8.721030,1.516454,11.481840,0.965515,10.467518,114.785097,0.620,0.177778,0.0,0.0,1.0,3.818774,32.182371,2.903,11.085901,22.330345,10.054037,1.431349,12.062559,4.155205,14.983662,33
2,2,1.0,3.0,5.111,0.841,29.542489,-18.641202,28.779399,-9.484120,9.188822,0.900682,3.0,1.653938,6.056309,12.112457,21.673970,-560.911894,-2.168220e+01,600.468184,27.255194,8.393991,1.526180,4.851073,7.739914,1.536830,...,8.284979,0.763090,4.224249,7.249356,1.510281,10.912497,0.920572,8.815708,75.309448,0.565,0.166667,0.0,0.0,1.0,0.707645,35.062049,7.039,4.981112,0.830186,15.136704,0.096832,4.797805,0.681603,3.544896,33
3,3,1.0,3.0,2.300,1.928,31.831760,-24.309871,29.433476,-13.844635,10.736523,0.667358,3.0,1.796073,0.747730,7.267474,12.112457,-346.354540,-1.580947e-11,352.184263,2.477557,8.175966,0.000000,3.597425,6.758798,1.483732,...,10.301717,1.335408,5.586910,9.129828,1.484540,12.960691,0.913245,7.622301,68.682652,0.620,0.166667,0.0,0.0,1.0,2.566353,37.724248,3.834,9.839397,101.158479,13.998713,0.739862,10.875639,2.836630,37.630952,33
4,4,1.0,3.0,1.931,1.534,27.362232,-27.689270,26.708155,-14.716738,12.989011,0.845204,3.0,1.621174,6.056229,9.910228,15.416763,-438.017217,-3.195719e+01,484.494892,0.736572,12.427468,2.670815,7.576395,11.500858,1.485348,...,15.697854,4.878326,9.838412,14.362446,1.544821,17.279794,0.974440,13.037398,-12.331252,0.650,0.155556,0.0,0.0,1.0,4.846472,48.089117,3.150,15.266386,90.821979,14.314349,1.960793,19.455974,6.176499,36.056311,33


# 欠損値とかもろもろ

## expdata_doc

In [5]:
# 欠損値のカウント: Trueは欠損値
df_fm.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,Unnamed: 0,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,...,ete_larg_deviation_ave,20_ete_line_ave,50_ete_line_ave,80_ete_line_ave,ave_direction_ave,length_trajectory_ave,ratio_ete_ave,ave_v_ave,5points_m_acc_ave,m_stroke_press_ave,m_stroke_area_cover_ave,finger_orien_ave,cd_finger_orien_ave,phone_orien_ave,2stroke_a,2stroke_distance,2stroke_time,2stroke_v,a_stroke_inter,d_stroke_inter,outside_a,outside_d,outside_v,v_stroke_inter,multi_flag
False,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,...,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399


In [6]:
df_fm_t = df_fm.drop({'Unnamed: 0', 'flag', 'finger_orien', 'cd_finger_orien', 'phone_orien',
                                   'user2', 'doc2', 'flag2', 'finger_orien2', 'cd_finger_orien2', 'phone_orien2',
                                   'user_ave', 'doc_ave', 'flag_ave', 'finger_orien_ave', 'cd_finger_orien_ave',
                                   'phone_orien_ave'}, axis=1)
df_fm_t.head()

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,...,50_pairwise_acc_ave,80_pairwise_acc_ave,3ots_m_v_ave,ete_larg_deviation_ave,20_ete_line_ave,50_ete_line_ave,80_ete_line_ave,ave_direction_ave,length_trajectory_ave,ratio_ete_ave,ave_v_ave,5points_m_acc_ave,m_stroke_press_ave,m_stroke_area_cover_ave,2stroke_a,2stroke_distance,2stroke_time,2stroke_v,a_stroke_inter,d_stroke_inter,outside_a,outside_d,outside_v,v_stroke_inter,multi_flag
0,27,6,2.118,1.398,6.5516,-36.286,7.1563,-15.825,20.4700,0.90037,1.5412,9.0436,14.3650,25.879,-252.21,-5.8162,233.88,7.3135,1.30430,0.205970,0.55933,9.644200e-01,1.5704,21.4680,0.95351,...,-31.1416,330.75,6.94350,1.131850,0.258005,0.482420,0.840140,1.47840,17.56500,0.945310,15.3965,16.083,0.520,0.044444,6.690606,60.335949,3.003,20.091891,52.205705,27.063437,0.946163,8.532505,2.841327,37.588107,33
1,27,6,2.142,0.885,12.0950,-42.434,11.8940,-29.633,12.8020,0.80483,1.5865,6.5276,13.1470,25.325,-481.51,-56.4670,427.62,6.5735,0.95940,0.310040,0.40551,7.158600e-01,1.3864,13.6620,0.93711,...,-28.2335,346.60,5.22510,0.627065,0.088795,0.259100,0.475475,1.34190,11.51275,0.937925,12.6055,134.970,0.550,0.044444,3.186922,30.626319,3.100,9.879458,5.717478,9.033896,1.324906,12.732347,4.107209,7.186870,33
2,27,6,2.943,0.958,13.6070,-38.503,13.0020,-29.734,8.7899,0.71704,1.6397,0.0000,8.3994,18.372,-405.22,0.0000,265.58,3.8767,0.29473,-0.132450,0.11269,2.350900e-01,1.2974,9.3635,0.93874,...,-24.4775,445.10,6.61375,-0.444235,-0.611825,-0.257180,0.117545,1.38445,10.53775,0.934190,12.4525,461.530,0.530,0.044444,2.073913,28.653367,3.717,7.708735,2.277958,8.975666,0.792000,10.942343,2.943864,4.521746,33
3,27,6,1.537,0.774,14.9170,-38.503,14.7160,-27.617,10.8880,0.86631,1.5893,7.4564,11.9870,20.518,-491.98,-48.9550,624.62,9.3508,-1.18320,-1.091200,-0.62705,2.498000e-17,1.4715,11.7120,0.92964,...,-53.4515,848.51,7.78345,-0.452005,-0.562763,-0.254025,0.112130,1.48550,10.88700,0.906030,14.1505,305.025,0.495,0.044444,5.925048,31.370763,2.301,13.633535,19.931940,11.603758,1.609201,8.520078,3.702772,15.208070,33
4,27,6,1.510,0.764,16.9330,-39.007,16.5300,-30.137,8.8790,0.69747,1.6162,5.1193,10.3960,31.476,-726.69,-57.9480,1072.40,6.2161,0.27919,-0.034326,0.11900,2.242600e-01,1.4995,10.0620,0.88242,...,-90.3140,918.27,6.67510,0.295425,-0.093408,0.065065,0.249595,1.46490,8.41280,0.913520,11.0941,-317.765,0.510,0.044444,4.571208,23.347901,2.260,10.330930,14.518423,8.079735,1.436157,7.335317,3.245716,10.830743,33


In [7]:
# 欠損値のカウント: Trueは欠損値
df_fm_t.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,...,50_pairwise_acc_ave,80_pairwise_acc_ave,3ots_m_v_ave,ete_larg_deviation_ave,20_ete_line_ave,50_ete_line_ave,80_ete_line_ave,ave_direction_ave,length_trajectory_ave,ratio_ete_ave,ave_v_ave,5points_m_acc_ave,m_stroke_press_ave,m_stroke_area_cover_ave,2stroke_a,2stroke_distance,2stroke_time,2stroke_v,a_stroke_inter,d_stroke_inter,outside_a,outside_d,outside_v,v_stroke_inter,multi_flag
False,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,...,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399


In [8]:
# 平均したものを削除
df_fm_t2 = df_fm_t.drop({'stroke_inter_ave', 'stroke_duration_ave', 'start_x_ave', 'start_y_ave',
                             'stop_x_ave', 'stop_y_ave', 'direct_ete_distance_ave', 'mean_result_leng_ave',
                             'direct_ete_line_ave', '20_pairwise_v_ave', '50_pairwise_v_ave', '80_pairwise_v_ave',
                             '20_pairwise_acc_ave', '50_pairwise_acc_ave', '80_pairwise_acc_ave', '3ots_m_v_ave',
                             'ete_larg_deviation_ave', '20_ete_line_ave', '50_ete_line_ave', '80_ete_line_ave',
                             'ave_direction_ave', 'length_trajectory_ave', 'ratio_ete_ave', 'ave_v_ave',
                             '5points_m_acc_ave', 'm_stroke_press_ave', 'm_stroke_area_cover_ave'}, axis=1)

df_fm_t2.head()

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,...,50_pairwise_acc2,80_pairwise_acc2,3ots_m_v2,ete_larg_deviation2,20_ete_line2,50_ete_line2,80_ete_line2,ave_direction2,length_trajectory2,ratio_ete2,ave_v2,5points_m_acc2,m_stroke_press2,m_stroke_area_cover2,2stroke_a,2stroke_distance,2stroke_time,2stroke_v,a_stroke_inter,d_stroke_inter,outside_a,outside_d,outside_v,v_stroke_inter,multi_flag
0,27,6,2.118,1.398,6.5516,-36.286,7.1563,-15.825,20.4700,0.90037,1.5412,9.0436,14.3650,25.879,-252.21,-5.8162,233.88,7.3135,1.30430,0.205970,0.55933,9.644200e-01,1.5704,21.4680,0.95351,...,-56.467,427.62,6.5735,0.95940,0.310040,0.405510,7.158600e-01,1.3864,13.6620,0.93711,15.4370,129.61,0.54,0.044444,6.690606,60.335949,3.003,20.091891,52.205705,27.063437,0.946163,8.532505,2.841327,37.588107,33
1,27,6,2.142,0.885,12.0950,-42.434,11.8940,-29.633,12.8020,0.80483,1.5865,6.5276,13.1470,25.325,-481.51,-56.4670,427.62,6.5735,0.95940,0.310040,0.40551,7.158600e-01,1.3864,13.6620,0.93711,...,0.000,265.58,3.8767,0.29473,-0.132450,0.112690,2.350900e-01,1.2974,9.3635,0.93874,9.7740,140.33,0.56,0.044444,3.186922,30.626319,3.100,9.879458,5.717478,9.033896,1.324906,12.732347,4.107209,7.186870,33
2,27,6,2.943,0.958,13.6070,-38.503,13.0020,-29.734,8.7899,0.71704,1.6397,0.0000,8.3994,18.372,-405.22,0.0000,265.58,3.8767,0.29473,-0.132450,0.11269,2.350900e-01,1.2974,9.3635,0.93874,...,-48.955,624.62,9.3508,-1.18320,-1.091200,-0.627050,2.498000e-17,1.4715,11.7120,0.92964,15.1310,782.73,0.50,0.044444,2.073913,28.653367,3.717,7.708735,2.277958,8.975666,0.792000,10.942343,2.943864,4.521746,33
3,27,6,1.537,0.774,14.9170,-38.503,14.7160,-27.617,10.8880,0.86631,1.5893,7.4564,11.9870,20.518,-491.98,-48.9550,624.62,9.3508,-1.18320,-1.091200,-0.62705,2.498000e-17,1.4715,11.7120,0.92964,...,-57.948,1072.40,6.2161,0.27919,-0.034326,0.119000,2.242600e-01,1.4995,10.0620,0.88242,13.1700,-172.68,0.49,0.044444,5.925048,31.370763,2.301,13.633535,19.931940,11.603758,1.609201,8.520078,3.702772,15.208070,33
4,27,6,1.510,0.764,16.9330,-39.007,16.5300,-30.137,8.8790,0.69747,1.6162,5.1193,10.3960,31.476,-726.69,-57.9480,1072.40,6.2161,0.27919,-0.034326,0.11900,2.242600e-01,1.4995,10.0620,0.88242,...,-122.680,764.14,7.1341,0.31166,-0.152490,0.011131,2.749300e-01,1.4303,6.7636,0.94462,9.0182,-462.85,0.53,0.044444,4.571208,23.347901,2.260,10.330930,14.518423,8.079735,1.436157,7.335317,3.245716,10.830743,33


In [9]:
# 欠損値のカウント: Trueは欠損値
df_fm_t2.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,...,50_pairwise_acc2,80_pairwise_acc2,3ots_m_v2,ete_larg_deviation2,20_ete_line2,50_ete_line2,80_ete_line2,ave_direction2,length_trajectory2,ratio_ete2,ave_v2,5points_m_acc2,m_stroke_press2,m_stroke_area_cover2,2stroke_a,2stroke_distance,2stroke_time,2stroke_v,a_stroke_inter,d_stroke_inter,outside_a,outside_d,outside_v,v_stroke_inter,multi_flag
False,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,...,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399,20399


In [12]:
df_fm_t2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20399 entries, 0 to 20398
Data columns (total 67 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user                  20399 non-null  int64  
 1   doc                   20399 non-null  int64  
 2   stroke_inter          20399 non-null  float64
 3   stroke_duration       20399 non-null  float64
 4   start_x               20399 non-null  float64
 5   start_y               20399 non-null  float64
 6   stop_x                20399 non-null  float64
 7   stop_y                20399 non-null  float64
 8   direct_ete_distance   20399 non-null  float64
 9   mean_result_leng      20399 non-null  float64
 10  direct_ete_line       20399 non-null  float64
 11  20_pairwise_v         20399 non-null  float64
 12  50_pairwise_v         20399 non-null  float64
 13  80_pairwise_v         20399 non-null  float64
 14  20_pairwise_acc       20399 non-null  float64
 15  50_pairwise_acc    

# expdata_fu_doc.csv

In [11]:
# 欠損値のカウント: Trueは欠損値
df_ffc.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,Unnamed: 0,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,...,ete_larg_deviation_ave,20_ete_line_ave,50_ete_line_ave,80_ete_line_ave,ave_direction_ave,length_trajectory_ave,ratio_ete_ave,ave_v_ave,5points_m_acc_ave,m_stroke_press_ave,m_stroke_area_cover_ave,finger_orien_ave,cd_finger_orien_ave,phone_orien_ave,2stroke_a,2stroke_distance,2stroke_time,2stroke_v,a_stroke_inter,d_stroke_inter,outside_a,outside_d,outside_v,v_stroke_inter,multi_flag
False,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,...,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625


In [13]:
df_ffc_t = df_ffc.drop({'Unnamed: 0', 'flag', 'finger_orien', 'cd_finger_orien', 'phone_orien',
                                   'user2', 'doc2', 'flag2', 'finger_orien2', 'cd_finger_orien2', 'phone_orien2',
                                   'user_ave', 'doc_ave', 'flag_ave', 'finger_orien_ave', 'cd_finger_orien_ave',
                                   'phone_orien_ave'}, axis=1)
df_ffc_t.head()

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,...,50_pairwise_acc_ave,80_pairwise_acc_ave,3ots_m_v_ave,ete_larg_deviation_ave,20_ete_line_ave,50_ete_line_ave,80_ete_line_ave,ave_direction_ave,length_trajectory_ave,ratio_ete_ave,ave_v_ave,5points_m_acc_ave,m_stroke_press_ave,m_stroke_area_cover_ave,2stroke_a,2stroke_distance,2stroke_time,2stroke_v,a_stroke_inter,d_stroke_inter,outside_a,outside_d,outside_v,v_stroke_inter,multi_flag
0,1.0,3.0,3.371,1.620,25.072961,-37.391416,26.926180,-28.997425,8.596134,0.784513,1.353503,0.000000,6.412537,22.981771,-89.228250,5.329094e-12,377.208959,3.406645,6.649785,-5.777682,-1.253648,-0.381545,1.044725,9.049638,0.949887,...,5.241771,399.337200,2.666870,8.612017,-0.708584,3.352146,4.660300,1.270401,11.299389,0.952447,7.663386,32.986789,0.640,0.188889,1.309492,29.694878,4.762,6.235800,2.661197,8.159233,1.270477,28.810146,6.050010,4.659756,33
1,1.0,3.0,2.062,1.391,30.087554,-21.475536,27.907296,-8.721030,12.939511,0.741063,1.740100,3.806625,7.267474,18.168866,-528.988537,1.048354e+01,421.465440,1.927095,10.574249,4.360515,7.957940,9.702146,1.496077,13.549140,0.955006,...,-5.599328,510.966812,14.591145,9.484120,2.943348,6.404506,8.721030,1.516454,11.481840,0.965515,10.467518,114.785097,0.620,0.177778,3.818774,32.182371,2.903,11.085901,22.330345,10.054037,1.431349,12.062559,4.155205,14.983662,33
2,1.0,3.0,5.111,0.841,29.542489,-18.641202,28.779399,-9.484120,9.188822,0.900682,1.653938,6.056309,12.112457,21.673970,-560.911894,-2.168220e+01,600.468184,27.255194,8.393991,1.526180,4.851073,7.739914,1.536830,9.414541,0.976024,...,-10.841099,476.326224,14.866376,8.284979,0.763090,4.224249,7.249356,1.510281,10.912497,0.920572,8.815708,75.309448,0.565,0.166667,0.707645,35.062049,7.039,4.981112,0.830186,15.136704,0.096832,4.797805,0.681603,3.544896,33
3,1.0,3.0,2.300,1.928,31.831760,-24.309871,29.433476,-13.844635,10.736523,0.667358,1.796073,0.747730,7.267474,12.112457,-346.354540,-1.580947e-11,352.184263,2.477557,8.175966,0.000000,3.597425,6.758798,1.483732,12.410454,0.865119,...,-15.978595,418.339578,1.607065,10.301717,1.335408,5.586910,9.129828,1.484540,12.960691,0.913245,7.622301,68.682652,0.620,0.166667,2.566353,37.724248,3.834,9.839397,101.158479,13.998713,0.739862,10.875639,2.836630,37.630952,33
4,1.0,3.0,1.931,1.534,27.362232,-27.689270,26.708155,-14.716738,12.989011,0.845204,1.621174,6.056229,9.910228,15.416763,-438.017217,-3.195719e+01,484.494892,0.736572,12.427468,2.670815,7.576395,11.500858,1.485348,13.510928,0.961371,...,-37.895046,698.561348,1.802662,15.697854,4.878326,9.838412,14.362446,1.544821,17.279794,0.974440,13.037398,-12.331252,0.650,0.155556,4.846472,48.089117,3.150,15.266386,90.821979,14.314349,1.960793,19.455974,6.176499,36.056311,33


In [14]:
# 欠損値のカウント: Trueは欠損値
df_ffc_t.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,...,50_pairwise_acc_ave,80_pairwise_acc_ave,3ots_m_v_ave,ete_larg_deviation_ave,20_ete_line_ave,50_ete_line_ave,80_ete_line_ave,ave_direction_ave,length_trajectory_ave,ratio_ete_ave,ave_v_ave,5points_m_acc_ave,m_stroke_press_ave,m_stroke_area_cover_ave,2stroke_a,2stroke_distance,2stroke_time,2stroke_v,a_stroke_inter,d_stroke_inter,outside_a,outside_d,outside_v,v_stroke_inter,multi_flag
False,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,...,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625


In [15]:
# 平均したものを削除
df_ffc_t2 = df_ffc_t.drop({'stroke_inter_ave', 'stroke_duration_ave', 'start_x_ave', 'start_y_ave',
                             'stop_x_ave', 'stop_y_ave', 'direct_ete_distance_ave', 'mean_result_leng_ave',
                             'direct_ete_line_ave', '20_pairwise_v_ave', '50_pairwise_v_ave', '80_pairwise_v_ave',
                             '20_pairwise_acc_ave', '50_pairwise_acc_ave', '80_pairwise_acc_ave', '3ots_m_v_ave',
                             'ete_larg_deviation_ave', '20_ete_line_ave', '50_ete_line_ave', '80_ete_line_ave',
                             'ave_direction_ave', 'length_trajectory_ave', 'ratio_ete_ave', 'ave_v_ave',
                             '5points_m_acc_ave', 'm_stroke_press_ave', 'm_stroke_area_cover_ave'}, axis=1)

df_ffc_t2.head()

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,...,50_pairwise_acc2,80_pairwise_acc2,3ots_m_v2,ete_larg_deviation2,20_ete_line2,50_ete_line2,80_ete_line2,ave_direction2,length_trajectory2,ratio_ete2,ave_v2,5points_m_acc2,m_stroke_press2,m_stroke_area_cover2,2stroke_a,2stroke_distance,2stroke_time,2stroke_v,a_stroke_inter,d_stroke_inter,outside_a,outside_d,outside_v,v_stroke_inter,multi_flag
0,1.0,3.0,3.371,1.620,25.072961,-37.391416,26.926180,-28.997425,8.596134,0.784513,1.353503,0.000000,6.412537,22.981771,-89.228250,5.329094e-12,377.208959,3.406645,6.649785,-5.777682,-1.253648,-0.381545,1.044725,9.049638,0.949887,...,1.048354e+01,421.465440,1.927095,10.574249,4.360515,7.957940,9.702146,1.496077,13.549140,0.955006,9.740576,78.951296,0.66,0.200000,1.309492,29.694878,4.762,6.235800,2.661197,8.159233,1.270477,28.810146,6.050010,4.659756,33
1,1.0,3.0,2.062,1.391,30.087554,-21.475536,27.907296,-8.721030,12.939511,0.741063,1.740100,3.806625,7.267474,18.168866,-528.988537,1.048354e+01,421.465440,1.927095,10.574249,4.360515,7.957940,9.702146,1.496077,13.549140,0.955006,...,-2.168220e+01,600.468184,27.255194,8.393991,1.526180,4.851073,7.739914,1.536830,9.414541,0.976024,11.194459,150.618897,0.58,0.155556,3.818774,32.182371,2.903,11.085901,22.330345,10.054037,1.431349,12.062559,4.155205,14.983662,33
2,1.0,3.0,5.111,0.841,29.542489,-18.641202,28.779399,-9.484120,9.188822,0.900682,1.653938,6.056309,12.112457,21.673970,-560.911894,-2.168220e+01,600.468184,27.255194,8.393991,1.526180,4.851073,7.739914,1.536830,9.414541,0.976024,...,-1.580947e-11,352.184263,2.477557,8.175966,0.000000,3.597425,6.758798,1.483732,12.410454,0.865119,6.436957,0.000000,0.55,0.177778,0.707645,35.062049,7.039,4.981112,0.830186,15.136704,0.096832,4.797805,0.681603,3.544896,33
3,1.0,3.0,2.300,1.928,31.831760,-24.309871,29.433476,-13.844635,10.736523,0.667358,1.796073,0.747730,7.267474,12.112457,-346.354540,-1.580947e-11,352.184263,2.477557,8.175966,0.000000,3.597425,6.758798,1.483732,12.410454,0.865119,...,-3.195719e+01,484.494892,0.736572,12.427468,2.670815,7.576395,11.500858,1.485348,13.510928,0.961371,8.807644,137.365303,0.69,0.155556,2.566353,37.724248,3.834,9.839397,101.158479,13.998713,0.739862,10.875639,2.836630,37.630952,33
4,1.0,3.0,1.931,1.534,27.362232,-27.689270,26.708155,-14.716738,12.989011,0.845204,1.621174,6.056229,9.910228,15.416763,-438.017217,-3.195719e+01,484.494892,0.736572,12.427468,2.670815,7.576395,11.500858,1.485348,13.510928,0.961371,...,-4.383290e+01,912.627804,2.868752,18.968240,7.085837,12.100429,17.224034,1.604295,21.048660,0.987510,17.267152,-162.027807,0.61,0.155556,4.846472,48.089117,3.150,15.266386,90.821979,14.314349,1.960793,19.455974,6.176499,36.056311,33


In [16]:
# 欠損値のカウント: Trueは欠損値
df_ffc_t2.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,...,50_pairwise_acc2,80_pairwise_acc2,3ots_m_v2,ete_larg_deviation2,20_ete_line2,50_ete_line2,80_ete_line2,ave_direction2,length_trajectory2,ratio_ete2,ave_v2,5points_m_acc2,m_stroke_press2,m_stroke_area_cover2,2stroke_a,2stroke_distance,2stroke_time,2stroke_v,a_stroke_inter,d_stroke_inter,outside_a,outside_d,outside_v,v_stroke_inter,multi_flag
False,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,...,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625,19625


In [17]:
df_ffc_t2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19625 entries, 0 to 19624
Data columns (total 67 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user                  19625 non-null  float64
 1   doc                   19625 non-null  float64
 2   stroke_inter          19625 non-null  float64
 3   stroke_duration       19625 non-null  float64
 4   start_x               19625 non-null  float64
 5   start_y               19625 non-null  float64
 6   stop_x                19625 non-null  float64
 7   stop_y                19625 non-null  float64
 8   direct_ete_distance   19625 non-null  float64
 9   mean_result_leng      19625 non-null  float64
 10  direct_ete_line       19625 non-null  float64
 11  20_pairwise_v         19625 non-null  float64
 12  50_pairwise_v         19625 non-null  float64
 13  80_pairwise_v         19625 non-null  float64
 14  20_pairwise_acc       19625 non-null  float64
 15  50_pairwise_acc    